In [1]:
from google.colab import drive
drive.mount('/content/drive')

!pip install -q transformers biopython kaggle
print("✅ Đã cài đặt xong thư viện!")

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 54.2 MB/s eta 0:00:00
✅ Đã cài đặt xong thư viện!


In [2]:
import os
from google.colab import files

# Upload file kaggle.json
print("Vui lòng upload file kaggle.json của bạn:")
files.upload()

# Cấu hình Kaggle API
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Vui lòng upload file kaggle.json của bạn:


Saving kaggle.json to kaggle.json


In [3]:
!kaggle datasets download -d seddiktrk/cafa6-protein-embeddings-esm2

# 3. Giải nén vào thư mục tên là 'esm2_data'
!unzip -q cafa6-protein-embeddings-esm2.zip -d /content/esm2_data

print("✅ Đã tải và giải nén xong vào folder: /content/esm2_data")

Dataset URL: https://www.kaggle.com/datasets/seddiktrk/cafa6-protein-embeddings-esm2
License(s): apache-2.0
 96% 1.22G/1.27G [00:11<00:01, 36.7MB/s]
100% 1.27G/1.27G [00:11<00:00, 120MB/s] 
✅ Đã tải và giải nén xong vào folder: /content/esm2_data


In [4]:
import numpy as np
import pandas as pd
import os
import gc
from tqdm.auto import tqdm

# ==============================================================================
# 1. CẤU HÌNH ĐƯỜNG DẪN
# ==============================================================================
# Folder chứa Ankh (TEMPLATE - Dùng để lấy thứ tự chuẩn)
ANKH_DIR = '/content/drive/MyDrive/CAFA6_Embeddings/Final_Merged'

# Folder chứa ESM2 gốc (SOURCE - Dữ liệu thô tải từ Kaggle)
ESM_SOURCE_DIR = '/content/esm2_data'

# Folder đầu ra (DESTINATION - Nơi lưu file ESM2 đã cắt nhỏ và sắp xếp)
OUTPUT_DIR = '/content/drive/MyDrive/CAFA6_Embeddings/ESM2_Aligned_Ankh'
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"🚀 BẮT ĐẦU QUÁ TRÌNH ALIGNMENT")
print(f"   - Template (Ankh): {ANKH_DIR}")
print(f"   - Source (ESM2):   {ESM_SOURCE_DIR}")
print(f"   - Output:          {OUTPUT_DIR}")

# ==============================================================================
# 2. HÀM XỬ LÝ ID (CLEAN ID)
# ==============================================================================
def clean_id(raw_id):
    """Làm sạch ID để khớp giữa Ankh (thường sạch) và ESM (thường lẫn tạp)"""
    # Xử lý nếu là bytes
    if isinstance(raw_id, bytes):
        s = raw_id.decode('utf-8')
    else:
        s = str(raw_id)

    # Xử lý b'...'
    if s.startswith("b'") or s.startswith('b"'):
        s = s[2:-1]

    # Cắt bỏ phần thừa (sp|ID|Name -> ID)
    if '|' in s:
        return s.split('|')[1]

    # Cắt bỏ phần space (ID Taxon -> ID)
    return s.split(' ')[0]

# ==============================================================================
# 3. TẠO TỪ ĐIỂN MAP: ESM ID -> INDEX
# ==============================================================================
print("\n[1/4] Đang đọc và đánh chỉ mục (Indexing) file ESM2 gốc...")

# Đọc file CSV chứa ID của ESM2
esm_ids_df = pd.read_csv(f"{ESM_SOURCE_DIR}/protein_ids.csv")
raw_esm_ids = esm_ids_df['protein_id'].tolist()

print(f"   -> Tổng số protein trong ESM2 gốc: {len(raw_esm_ids):,}")

# Tạo Map: {Cleaned_ID : Index_trong_file_npy}
esm_id_to_idx = {}
for idx, raw_id in enumerate(tqdm(raw_esm_ids, desc="Creating Index Map")):
    clean = clean_id(raw_id)
    esm_id_to_idx[clean] = idx

# Giải phóng RAM
del esm_ids_df, raw_esm_ids
gc.collect()

# ==============================================================================
# 4. HÀM TRÍCH XUẤT VÀ LƯU FILE
# ==============================================================================
def process_and_save(ankh_id_file, output_embed_name, output_id_name):
    print(f"\n🔄 Đang xử lý: {os.path.basename(ankh_id_file)} ...")

    # 1. Load Ankh IDs (Thứ tự chuẩn)
    target_ids = np.load(ankh_id_file, allow_pickle=True)
    print(f"   -> Số lượng ID cần lấy: {len(target_ids):,}")

    # 2. Tìm Index tương ứng bên ESM2
    valid_indices = []
    valid_ids = []
    missing_count = 0

    for pid in tqdm(target_ids, desc="Mapping IDs"):
        # Ankh ID thường đã sạch, nhưng clean lại cho chắc
        pid_clean = clean_id(pid)

        if pid_clean in esm_id_to_idx:
            valid_indices.append(esm_id_to_idx[pid_clean])
            valid_ids.append(pid_clean) # Lưu ID đã clean
        else:
            missing_count += 1

    print(f"   -> Tìm thấy: {len(valid_indices):,} | Bị thiếu: {missing_count}")

    if len(valid_indices) == 0:
        print("❌ LỖI: Không tìm thấy ID nào trùng khớp!")
        return

    # 3. Trích xuất Embedding từ file lớn (Dùng mmap để tiết kiệm RAM)
    # Load file ESM gốc ở chế độ read-only
    print("   ⏳ Đang trích xuất Embeddings (mmap)...")
    source_embeds = np.load(f"{ESM_SOURCE_DIR}/protein_embeddings.npy", mmap_mode='r')

    # Dùng Fancy Indexing để lấy các dòng cần thiết
    # Lưu ý: Bước này sẽ copy dữ liệu từ đĩa vào RAM -> Tạo array mới
    extracted_embeds = source_embeds[valid_indices]

    # 4. Lưu ra file mới
    print(f"   💾 Đang lưu file vào {OUTPUT_DIR}...")
    np.save(f"{OUTPUT_DIR}/{output_embed_name}", extracted_embeds)
    np.save(f"{OUTPUT_DIR}/{output_id_name}", np.array(valid_ids))

    print(f"   ✅ Xong: {extracted_embeds.shape}")

    # Dọn dẹp
    del extracted_embeds, valid_indices, valid_ids, source_embeds
    gc.collect()

# ==============================================================================
# 5. CHẠY THỰC TẾ
# ==============================================================================

# --- Xử lý TRAIN ---
process_and_save(
    ankh_id_file=f"{ANKH_DIR}/ankh_train_FINAL_ids.npy",
    output_embed_name="esm2_train_FINAL_embeddings.npy",
    output_id_name="esm2_train_FINAL_ids.npy"
)

# --- Xử lý TEST ---
process_and_save(
    ankh_id_file=f"{ANKH_DIR}/ankh_test_FINAL_ids.npy",
    output_embed_name="esm2_test_FINAL_embeddings.npy",
    output_id_name="esm2_test_FINAL_ids.npy"
)

print("\n" + "="*50)
print("🎉 HOÀN TẤT! Dữ liệu ESM2 đã được đồng bộ theo Ankh.")
print(f"📂 Folder kết quả: {OUTPUT_DIR}")

🚀 BẮT ĐẦU QUÁ TRÌNH ALIGNMENT
   - Template (Ankh): /content/drive/MyDrive/CAFA6_Embeddings/Final_Merged
   - Source (ESM2):   /content/esm2_data
   - Output:          /content/drive/MyDrive/CAFA6_Embeddings/ESM2_Aligned_Ankh

[1/4] Đang đọc và đánh chỉ mục (Indexing) file ESM2 gốc...
   -> Tổng số protein trong ESM2 gốc: 287,001


Creating Index Map:   0%|          | 0/287001 [00:00<?, ?it/s]


🔄 Đang xử lý: ankh_train_FINAL_ids.npy ...
   -> Số lượng ID cần lấy: 82,404


Mapping IDs:   0%|          | 0/82404 [00:00<?, ?it/s]

   -> Tìm thấy: 82,404 | Bị thiếu: 0
   ⏳ Đang trích xuất Embeddings (mmap)...
   💾 Đang lưu file vào /content/drive/MyDrive/CAFA6_Embeddings/ESM2_Aligned_Ankh...
   ✅ Xong: (82404, 1280)

🔄 Đang xử lý: ankh_test_FINAL_ids.npy ...
   -> Số lượng ID cần lấy: 224,309


Mapping IDs:   0%|          | 0/224309 [00:00<?, ?it/s]

   -> Tìm thấy: 224,309 | Bị thiếu: 0
   ⏳ Đang trích xuất Embeddings (mmap)...
   💾 Đang lưu file vào /content/drive/MyDrive/CAFA6_Embeddings/ESM2_Aligned_Ankh...
   ✅ Xong: (224309, 1280)

🎉 HOÀN TẤT! Dữ liệu ESM2 đã được đồng bộ theo Ankh.
📂 Folder kết quả: /content/drive/MyDrive/CAFA6_Embeddings/ESM2_Aligned_Ankh


In [5]:
import numpy as np

# ==============================================================================
# CẤU HÌNH ĐƯỜNG DẪN (Thay đổi folder của bạn vào đây)
# ==============================================================================
# Folder chứa file ID của Ankh (File chuẩn)
ANKH_DIR = '/content/drive/MyDrive/CAFA6_Embeddings/Final_Merged'

# Folder chứa file ID của ESM2 (File cần check)
ESM_DIR = '/content/drive/MyDrive/CAFA6_Embeddings/ESM2_Aligned_Ankh'
# Hoặc nếu bạn check file gốc chưa align thì trỏ vào folder gốc:
# ESM_DIR = '/content/esm2_data'

# Tên file
FILE_1_NAME = 'ankh_train_FINAL_ids.npy'
FILE_2_NAME = 'esm2_train_FINAL_ids.npy'
# Lưu ý: Nếu check file gốc ESM thì tên có thể là 'protein_ids.csv' -> Code dưới xử lý .npy là chính

PATH_1 = f"{ANKH_DIR}/{FILE_1_NAME}"
PATH_2 = f"{ESM_DIR}/{FILE_2_NAME}"

# ==============================================================================
# HÀM LOAD VÀ CHECK
# ==============================================================================
def check_alignment(path1, path2):
    print(f"📂 Đang so sánh:\n   1. {path1}\n   2. {path2}\n")

    try:
        # Load file 1
        ids1 = np.load(path1, allow_pickle=True)

        # Load file 2
        ids2 = np.load(path2, allow_pickle=True)

        # In ra 5 dòng đầu
        print("-" * 40)
        print(f"👀 5 ID ĐẦU TIÊN CỦA FILE 1 (Tổng: {len(ids1):,}):")
        print(ids1[:5])

        print("-" * 40)
        print(f"👀 5 ID ĐẦU TIÊN CỦA FILE 2 (Tổng: {len(ids2):,}):")
        print(ids2[:5])
        print("-" * 40)

        # Kiểm tra đồng bộ
        print("\n🔍 KẾT QUẢ KIỂM TRA:")

        # 1. Check số lượng
        if len(ids1) != len(ids2):
            print(f"❌ SAI SỐ LƯỢNG: File 1 có {len(ids1)} dòng, File 2 có {len(ids2)} dòng.")
            return

        # 2. Check nội dung (Từng phần tử)
        if np.array_equal(ids1, ids2):
            print("✅ ĐỒNG BỘ HOÀN HẢO! (100% Khớp nhau)")
        else:
            print("⚠️ CẢNH BÁO: Cùng số lượng nhưng KHÁC NỘI DUNG hoặc SAI THỨ TỰ!")

            # Tìm vị trí sai đầu tiên
            mismatch_idx = np.where(ids1 != ids2)[0]
            first_fail = mismatch_idx[0]
            print(f"   -> Sai lệch bắt đầu tại index: {first_fail}")
            print(f"   -> File 1: {ids1[first_fail]}")
            print(f"   -> File 2: {ids2[first_fail]}")

    except Exception as e:
        print(f"❌ Lỗi khi đọc file: {e}")

# ==============================================================================
# CHẠY
# ==============================================================================
check_alignment(PATH_1, PATH_2)

📂 Đang so sánh:
   1. /content/drive/MyDrive/CAFA6_Embeddings/Final_Merged/ankh_train_FINAL_ids.npy
   2. /content/drive/MyDrive/CAFA6_Embeddings/ESM2_Aligned_Ankh/esm2_train_FINAL_ids.npy

----------------------------------------
👀 5 ID ĐẦU TIÊN CỦA FILE 1 (Tổng: 82,404):
['A0JNW5' 'A6BM72' 'A6H8Y1' 'A6NGG8' 'A6NKB5']
----------------------------------------
👀 5 ID ĐẦU TIÊN CỦA FILE 2 (Tổng: 82,404):
['A0JNW5' 'A6BM72' 'A6H8Y1' 'A6NGG8' 'A6NKB5']
----------------------------------------

🔍 KẾT QUẢ KIỂM TRA:
✅ ĐỒNG BỘ HOÀN HẢO! (100% Khớp nhau)
